In [1]:
# conda install opencv

In [4]:
import pandas as pd
import numpy as np

In [9]:
import glob
import cv2

images = [cv2.imread(file) for file in glob.glob('./data/train/nowildfire/*.jpg')]

Premature end of JPEG file


In [12]:
len(images)

14500

In [14]:
images

[array([[[182, 190, 189],
         [207, 215, 214],
         [161, 169, 168],
         ...,
         [181, 193, 195],
         [180, 192, 194],
         [179, 191, 193]],
 
        [[182, 190, 189],
         [206, 214, 213],
         [161, 169, 168],
         ...,
         [181, 193, 195],
         [180, 192, 194],
         [179, 191, 193]],
 
        [[182, 190, 189],
         [205, 213, 212],
         [160, 168, 167],
         ...,
         [181, 193, 195],
         [179, 191, 193],
         [179, 191, 193]],
 
        ...,
 
        [[ 70,  67,  59],
         [ 71,  68,  60],
         [ 69,  66,  58],
         ...,
         [207, 219, 225],
         [204, 216, 222],
         [199, 211, 217]],
 
        [[ 69,  66,  58],
         [ 72,  69,  61],
         [ 68,  65,  57],
         ...,
         [207, 219, 225],
         [204, 216, 222],
         [199, 211, 217]],
 
        [[ 68,  65,  57],
         [ 73,  70,  62],
         [ 67,  64,  56],
         ...,
         [208, 220, 226],
  

In [17]:
import io
import os

WIDTH = 28
HEIGHT = 28

all_images = []
for image_path in os.listdir('./data/train/nowildfire/'):
  img = io.imread(image_path , as_grey=True)
  img = img.reshape([WIDTH, HEIGHT, 1])
  all_images.append(img)
X_train = np.array(all_images)



AttributeError: module 'io' has no attribute 'imread'

In [21]:
import keras
from keras.preprocessing.image import load_img

ModuleNotFoundError: No module named 'keras'

In [ ]:
# pip install keras

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist, cifar10

import keras
from keras.preprocessing.image import load_img

ModuleNotFoundError: No module named 'tensorflow'